https://docs.google.com/spreadsheets/d/10i4degomMeKCkGgJ9p_0hZkRF3w8vXL6OxgRTERUuvI/edit#gid=515265521

https://docs.google.com/spreadsheets/d/1ha9f6hSL-vD1J8XrOS3nHTYfWDAqLWWIGbyYhm_GWPE/edit#gid=1875032415


# 1. Quantumm

## 1.1 IOS

In [ ]:
import os
from tqdm.notebook import tqdm
os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
%run "Collect_data.py"
# All apps_id with test groups
query_appid = \
'''
SELECT appsflyer_id, event_time, JSONExtractString(event_value, 'Group')
FROM partners.lightcity
WHERE event_name  = 'SplitTest'
      AND app_id = 'id1489263669'
      AND JSONExtractRaw(event_value, 'TestName') = 
'''
apps_test = {"id1489263669": 
             ["BigPriceBefore", "StarterPackShowX3Value",
                                "LevelUpRew", "SkipAdsByDiamond",
                                "TrueNoAds", "TwoStarters"],
            "com.nopowerup.idlelightcity": 
             ["BigPriceBefore", "StarterPackShowX3Value",
                                "LevelUpRew", "SkipAdsByDiamond",
                                "TrueNoAds", "TwoStarters"]}
all_queries = []
for testnames in tqdm(apps_test['id1489263669']):
    query_testname = query_appid + "'" + '"' + testnames + '"' + "'"
    apps_test_id = clickhouse(query=query_testname, clickhouse_query=clickhouse_query)
    apps_test_id['test_name'] = testnames
    apps_test_id.to_csv("Data/Quantum_AB_historic/ARPU/IOS/Apps_id_info" + testnames + ".csv", index = False)
# %time apps_id = clickhouse(query=query_appid, clickhouse_query=clickhouse_query)
# 2 hours 16 mins

In [ ]:
# merge all files
import os
os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
%run "Collect_data.py"
# Import data 

from tqdm.notebook import tqdm
import glob

arpu_ios = []
for file in tqdm(glob.glob("Data/Quantum_AB_historic/ARPU/IOS/Apps_id_info/*.csv")):
    d = pd.read_csv(file)
    arpu_ios.append(d)
arpu_ios = pd.concat(arpu_ios)
arpu_ios.rename(columns = {"JSONExtractString(event_value, 'Group')":'group_test'},inplace=True)
arpu_ios['event_time'] = pd.to_datetime(arpu_ios['event_time']) 
arpu_ios['group_test'] = arpu_ios['group_test'].astype('category')
arpu_ios['test_name'] = arpu_ios['test_name'].astype('category')
arpu_ios.sort_values("event_time",  inplace=True)
arpu_ios.to_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/IOS/Apps_id_ios_arpu")

In [ ]:
for test in arpu["test_name"].unique():
    print(f"Minimum datetime for {test}: {arpu[arpu['test_name'] == test]['event_time'].min()} \n",
         f"Maximum datetime for {test}: {arpu[arpu['test_name'] == test]['event_time'].max()} \n",)

In [ ]:
%%time
import os
from tqdm.notebook import tqdm
os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
# %run "/Users/iakubovskii/AppBooster/Predictive_model/Import data.py"
# %run "Collect_data.py"
query_revenue = \
'''
SELECT if(has_conversion, revenue, 0) AS revenue, appsflyer_id
      FROM (
        SELECT
          appsflyer_id,
          sequenceMatch('(?1).*(?2)')(
            event_time, 
            event_name = 'SplitTest'
              AND JSONExtractRaw(event_value, 'TestName') = {}
              AND JSONExtractRaw(event_value, 'Group') = {},
            event_revenue_usd > 0) as has_conversion,
            sum(event_revenue_usd) as revenue
        FROM partners.lightcity
        GROUP BY appsflyer_id
      )
WHERE revenue > 0
'''
for test_name in tqdm(arpu['test_name'].unique()):
    test_name_df = []
    for group in tqdm(arpu[arpu['test_name'] == test_name]
                      ["JSONExtractString(event_value, 'Group')"].unique()):
        test_name_iter  = "'" + '"' + test_name + '"' + "'" # add test name
        group_name_iter  = "'" + '"' + group + '"' + "'" # add group of test
        query_revenue_test_group = query_revenue.format(test_name_iter, group_name_iter)
#         query_revenue_test_group = query_revenue_test_group + " IN " + str(
#             arpu[(arpu['test_name'] == test_name) &
#                  (arpu["JSONExtractString(event_value, 'Group')"] == group)]
#             ['appsflyer_id'].unique().tolist()).\
#                             replace("[", "(").replace("]", ")") # add list of apps id
        apps_revenue = clickhouse(query=query_revenue_test_group,  
                                          clickhouse_query=clickhouse_query) # go query
        apps_revenue['group_test'] = group # assign group in test
        test_name_df.append(apps_revenue) # append to data for every test 
    test_name_df = pd.concat(test_name_df) # concat
    test_name_df.to_csv("Data/Quantum_AB_historic/ARPU/Revenue/" + test_name + ".csv") # export to csv


# query_non_nul_revenue = \

# for test_name in tqdm(arpu['test_name'].unique()):
#     for group in tqdm(arpu[arpu['test_name'] == test_name]
#                       ["JSONExtractString(event_value, 'Group')"].unique()):
#         list_unique_apps_id = arpu[(arpu['test_name'] == test_name) &
#                  (arpu["JSONExtractString(event_value, 'Group')"] == group)]['appsflyer_id'].\
#                             unique().tolist()
        
        
# #         query_revenue_test_group = query_non_nul_revenue + " IN " + "(" +  + ")" + " SETTINGS max_parallel_replicas=5"
# #                             replace("[", "(").replace("]", ")") # add list of apps id

#         apps_revenue = clickhouse(query=query_revenue_test_group,  
#                                           clickhouse_query=clickhouse_query) # go query
    
# query_revenue_starter = query_revenue + " IN " + str(
#     starter_pack['appsflyer_id'].unique().tolist()).replace(
#     "[", "(").replace("]", ")")
# %time apps_revenue_b = clickhouse(query=query_revenue_starter, 
#                                           clickhouse_query=clickhouse_query)

In [ ]:
# Merge all revenues
arpu_revenue = []
for file in tqdm(glob.glob("Data/Quantum_AB_historic/ARPU/IOS/Revenue/*.csv")):
    d = pd.read_csv(file)
    d['test_name'] = file.split(".csv")[0].split("/")[5]
    arpu_revenue.append(d)
arpu_revenue = pd.concat(arpu_revenue)
arpu_revenue.drop("Unnamed: 0", axis=1, inplace=True)
arpu_revenue['group_test'] = arpu_revenue['group_test'].astype("category")
arpu_revenue['test_name'] = arpu_revenue['test_name'].astype("category")
arpu_revenue.to_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Revenues_ios_android")

In [ ]:
# Merge revenues and apps id
apps_id_ios = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/IOS/Apps_id_ios_arpu")
revenues = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Revenues_ios_android")
revenue_ios = apps_id_ios.set_index(['appsflyer_id', 'group_test', 'test_name']).join(
revenues.set_index(['appsflyer_id', 'group_test', 'test_name'])).fillna(0).sort_values("event_time")
revenue_ios = revenue_ios.reset_index().drop_duplicates(
    ['appsflyer_id', 'group_test', 'test_name', 'revenue']).set_index(
    ['appsflyer_id', 'group_test', 'test_name'])
revenue_ios.columns = ['time_to_group', 'total_id_revenue']
revenue_ios.to_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/IOS/final_df")

In [ ]:
revenue_ios.shape
revenue_ios_new = revenue_ios.reset_index().drop_duplicates(
    ['appsflyer_id', 'group_test', 'test_name', 'revenue']).set_index(
    ['appsflyer_id', 'group_test', 'test_name'])

## 1.2 Android

In [ ]:
import os
from tqdm.notebook import tqdm
os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
%run "Collect_data.py"
# All apps_id with test groups
query_appid = \
'''
SELECT appsflyer_id, event_time, JSONExtractString(event_value, 'Group')
FROM partners.lightcity
WHERE event_name  = 'SplitTest'
      AND app_id = 'com.nopowerup.idlelightcity'
      AND JSONExtractRaw(event_value, 'TestName') = 
'''
apps_test = {"id1489263669": 
             ["BigPriceBefore", "StarterPackShowX3Value",
                                "LevelUpRew", "SkipAdsByDiamond",
                                "TrueNoAds", "TwoStarters"],
            "com.nopowerup.idlelightcity": 
             ["BigPriceBefore", "StarterPackShowX3Value",
                                "LevelUpRew", "SkipAdsByDiamond",
                                "TrueNoAds", "TwoStarters"]}
all_queries = []
for testnames in tqdm(apps_test['com.nopowerup.idlelightcity']):
    query_testname = query_appid + "'" + '"' + testnames + '"' + "'"
    apps_test_id = clickhouse(query=query_testname, clickhouse_query=clickhouse_query)
    apps_test_id['test_name'] = testnames
    apps_test_id.to_csv("Data/Quantum_AB_historic/ARPU/Android/Apps_id_info/" + testnames + ".csv", 
                        index = False)
# 1 hours 35 mins

In [ ]:
# merge all apps id files
# import os
# os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
# %run "Collect_data.py"
# %run "/Users/iakubovskii/AppBooster/Predictive_model/Import data.py"
# Import data 

from tqdm.notebook import tqdm
import glob

arpu_andr = []
for file in tqdm(glob.glob("Data/Quantum_AB_historic/ARPU/Android/Apps_id_info/*.csv")):
    d = pd.read_csv(file)
    arpu_andr.append(d)
arpu_andr = pd.concat(arpu_andr)
arpu_andr.rename(columns = {"JSONExtractString(event_value, 'Group')":'group_test'},inplace=True)
arpu_andr['event_time'] = pd.to_datetime(arpu_andr['event_time']) 
arpu_andr['group_test'] = arpu_andr['group_test'].astype('category')
arpu_andr['test_name'] = arpu_andr['test_name'].astype('category')
arpu_andr.sort_values("event_time", inplace=True)
arpu_andr.to_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Android/Apps_id_android_arpu")

In [ ]:
# Merge revenues and apps id
apps_id_android = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Android/Apps_id_android_arpu")
revenues = pd.read_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Revenues_ios_android")
revenue_android = apps_id_android.set_index(['appsflyer_id', 'group_test', 'test_name']).join(
revenues.set_index(['appsflyer_id', 'group_test', 'test_name'])).fillna(0).sort_values("event_time")
revenue_android = revenue_android.reset_index().drop_duplicates(
    ['appsflyer_id', 'group_test', 'test_name', 'revenue']).set_index(
    ['appsflyer_id', 'group_test', 'test_name'])
revenue_android.columns = ['time_to_group', 'total_id_revenue']
revenue_android.to_pickle("/Users/iakubovskii/AppBooster/AB_testing/Data/Quantum_AB_historic/ARPU/Android/final_df")

# 2. Appbooster 

http://51.158.122.91:8123/

reader

4CYXcXLQ4x

см. таблицу biggie_production.support_appquantum_com

In [ ]:
import os
from tqdm.notebook import tqdm
os.chdir("/Users/iakubovskii/AppBooster/AB_testing/")
%run "Collect_data.py"
%run "/Users/iakubovskii/AppBooster/Predictive_model/Import data.py"

In [ ]:
%%time
query_amount = '''
select app_id, 
    JSONExtractRaw(event_value, 'testName') as test_name, 
    JSONExtractRaw(event_value, 'variant') as variant,
    count(*) as amount 
from biggie_production.support_appquantum_com 
where event_name='ab_test'
group by app_id, test_name, variant
order by app_id, test_name, variant 
''' 
# ab_observes = clickhouse_query(query=query_amount,
#                       user="reader", password="4CYXcXLQ4x", host="51.158.122.91:8123")
# ab_observes = clickhouse_transform_pandas(ab_observes)

query_abtest = '''
select appsflyer_id, app_id, event_revenue_usd,
    JSONExtractRaw(event_value, 'testName') as test_name, 
    JSONExtractRaw(event_value, 'variant') as variant
from biggie_production.support_appquantum_com 
where event_name='ab_test'
''' 
ab_test = clickhouse_query(query=query_abtest,
                      user="reader", password="4CYXcXLQ4x", host="51.158.122.91:8123")
ab_test = optimize_pd_datatypes(pd.DataFrame.from_records(ab_test['data']))

In [ ]:
ab_test.info(memory_usage="deep")

In [ ]:
android = ab_test.query("platform == 'android'")
ios = ab_test.query("platform == 'ios'")

In [ ]:
ab_test['platform'].value_counts()

In [ ]:
ab_test.to_csv("/Users/iakubovskii/AppBooster/AB_testing/Data/AB_historic_other.csv", index=False)